# FLASH V3 cadCAD MODEL

![System Map](data/flashV3map.png)

In [41]:
%reload_ext autoreload
%autoreload 2
%reset -f
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from decimal import Decimal
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD import configs

# from model.parts.amm_cp import AmmCp

## SUPPORT METHODS

In [42]:
# ********************************************************************************************
# x = token1_amount                                                                         //
# y = token2_amount                 x * y = k                                               //
# k = constant_product_invariant                                                            //
# ********************************************************************************************

# ********************************************************************************************
# x = input_token_liquidity                                                                 //
# y = output_token_liquidity                  /    k    \                                   //
# k = constant_product_invariant    oA = y - | --------- |                                  //
# iA = input_amount                           \  x + iA /                                   //
# oA = output_amount                                                                        //
# ********************************************************************************************
def swap(
    input_amount: Decimal,
    input_token_liquidity: Decimal,
    output_token_liquidity: Decimal
) -> (Decimal, Decimal, Decimal):
    # Calculate the amount of output token to return
    k = input_token_liquidity * output_token_liquidity
    output_amount = output_token_liquidity - ((k / (input_token_liquidity + input_amount)))

    # Calculate updated liquidity calcs
    updated_input_token_liquidity = input_token_liquidity + input_amount
    updated_output_token_liquidity = output_token_liquidity - output_amount

    return (output_amount, updated_input_token_liquidity, updated_output_token_liquidity)

## cadCAD METHODS

#### POLICY UPDATE METHODS

In [43]:
#####
# External market random peer rate of return generator
def p_market_apy(_params, substep, state_history, state_current, **kwargs):
    random_rr = Decimal(random.randrange(_params['market_apy_rand_range'][0], _params['market_apy_rand_range'][1])) / Decimal(100)
    return {'market_apy': Decimal(random_rr)}

#####
# External AMM random swap amount generator
def p_ext_swap_amount(_params, substep, state_history, state_current, **kwargs):
    random_float = np.random.rand() * _params['ext_swap_rand_factor']
    return {'ext_swap_amount': Decimal(random_float)}

# External AMM random input token generator
def p_ext_swap_input_token(_params, substep, state_history, state_current, **kwargs):
    return {'ext_swap_input_token': random.sample(['flash', 'alt'], 1)[0]}

#####
# Internal AMM swap amount calculator
def p_int_swap_amount(_params, substep, state_history, state_current, **kwargs):
    (input_token, input_amount) = amm_arb_calc(state_current)
    return {'int_swap_amount': input_amount}

# Internal AMM input token calculator
def p_int_swap_input_token(_params, substep, state_history, state_current, **kwargs):
    (input_token, input_amount) = amm_arb_calc(state_current)
    return {'int_swap_input_token': input_token}

# If the External AMM FLASH price (ratio) is greater than the
# Internal AMM FLASH price (ratio), buy enough FLASH to equal
# the external ratio, otherwise sell enough to equal the ratio
# y = sqrt(k * px) | opposing_token_ending_total = sqrt(pair_invariant * token_target_price)
# return the string of the input token and decimal amount to input
def amm_arb_calc(state_current: dict) -> (str, Decimal):
    external_px = Decimal(state_current['ext_liquidity']['alt'] / state_current['ext_liquidity']['flash'])
    internal_px = Decimal(state_current['int_liquidity']['alt'] / state_current['int_liquidity']['flash'])
    internal_invariant = Decimal(state_current['int_liquidity']['alt'] * state_current['int_liquidity']['flash'])
    
    input_token = "alt"
    target_alt_ending_total = np.sqrt(internal_invariant * external_px)
    input_amount = target_alt_ending_total - Decimal(state_current['int_liquidity']['alt'])
    if internal_px > external_px:
        input_token = "flash"
        # We have the ending total for "y" (the opposing pair pool liquidity),
        # but we need to calculate the ending "x" (FLASH pair pool liquidity)
        # using the invariant and constant product formula
        flash_ending_total = internal_invariant / target_alt_ending_total
        input_amount = flash_ending_total - Decimal(state_current['int_liquidity']['flash'])
    
    return (input_token, Decimal(input_amount))

#####
# ********************************************************************************************
# fl2 = CP AMM ending FLASH liquidity                                                       //
# al1 = CP AMM beginning ALT liquidity                              al1                     //
# px1 = FLASH beginning price (ALT/FLASH)         fl2 = ----------------------------        //
# FPY = Flash (Annual) Percentage Yield                  sqrt( px1 - (FPY - mAPY) )         //
# mAPY = Market (peer) Annual Percentage Yield                                              //
# ********************************************************************************************
# FLASH stake opportunity calculator
def p_flash_to_stake(_params, substep, state_history, state_current, **kwargs):
    m_apy = state_current['market_apy'] # This will be the previous timestep's market rate of return
    fpy = state_current['policies']['mint_rate']
    
    # A new stake will be worth the opportunity if the stake rate of return
    # is greater than the competing market rate of return (on an annual basis)
    # The mint rate is the FLASH returned per flashALT LP staked PER DAY.
    # TODO: Do not assume staking for 1 year - vary based on perceived protocol volatility?
    if fpy > m_apy:
        # Mint rate is greater than the market rate, but we need to calculate
        # the amount of FLASH needed to produce price slippage enough to remove
        # all rate of return premium Flash provides over competing protocols.
        # See the mathematical proof for formula logic: fl2 = al1 / sqrt(px1 - (FPY - mAPY))
        # FLASH needed = fl2 - fl1
        al_1 = state_current['ext_liquidity']['alt']
        px_1 = state_current['ext_liquidity']['alt'] / state_current['ext_liquidity']['flash']
        fl_2 = al_1 / np.sqrt(px_1 - (fpy - m_apy))
        
        fl_1 = state_current['ext_liquidity']['flash']
        flash_needed = fl_2 - fl_1
        
        # TODO: Iron out calcs for LPs staked vs "FLASH" staked - always equal due to current block equal price?
        # FLASH minted = FLASH staked * FPY
        # FLASH minted / FPY = FLASH staked
        flash_to_stake = flash_needed / fpy
    return {'flash_to_stake': flash_to_stake}

#### STATE UPDATE METHODS

In [48]:
#####
# External market random peer rate of return saver
def s_ext_rr(_params, substep, state_history, state_current, _input, **kwargs):
    return {'market_apy': Decimal(_input['market_apy'])}

#####
# External AMM swap execution
def s_ext_swap(_params, substep, state_history, state_current, _input, **kwargs):
    
    # First, swap for randomized speculation
    input_token_name = _input['ext_swap_input_token']
    output_token_name = 'alt' if input_token_name == "flash" else 'flash'
    
    (output_amount,
        updated_input_token_liquidity,
        updated_output_token_liquidity
    ) = swap(
        input_amount=_input['ext_swap_amount'],
        input_token_liquidity=state_current['ext_liquidity'][input_token_name],
        output_token_liquidity=state_current['ext_liquidity'][output_token_name]
    )
    print("EXT AMM: output %f %s" % (output_amount, output_token_name))
    
    # Second, swap FLASH just minted (executed in parallel) for ALT
    # FLASH minted = FLASH staked * FPY
    fpy = state_current['policies']['mint_rate']
    flash_staked = _input['flash_to_stake']
    flash_minted = flash_staked * fpy
    
    # In this swap, the input is always FLASH, so determine which updated liquidity
    # from the first swap should be used for input/output
    second_input_token_liquidity = updated_input_token_liquidity
    second_output_token_liquidity = updated_output_token_liquidity
    if input_token_name == "alt":
        second_input_token_liquidity = updated_output_token_liquidity
        second_output_token_liquidity = updated_input_token_liquidity
    
    (output_amount,
        updated_flash_token_liquidity,
        updated_alt_token_liquidity
    ) = swap(
        input_amount=flash_minted,
        input_token_liquidity=second_input_token_liquidity,
        output_token_liquidity=second_output_token_liquidity
    )
    print("EXT AMM for STAKE: output %f alt" % (output_amount))
    
    # Finally, update the liquidity states after both swaps
    updated_ext_liquidity = {
        'alt': updated_alt_token_liquidity,
        'flash': updated_flash_token_liquidity
    }
    
#     updated_ext_liquidity = {
#         input_token_name: updated_input_token_liquidity,
#         output_token_name: updated_output_token_liquidity
#     }
    return ('ext_liquidity', updated_ext_liquidity)

#####
# Internal AMM swap execution
def s_int_swap(_params, substep, state_history, state_current, _input, **kwargs):
    
    # First, swap for randomized speculation
    input_token_name = _input['int_swap_input_token']
    output_token_name = 'alt' if input_token_name == "flash" else 'flash'
    
    (output_amount,
        updated_input_token_liquidity,
        updated_output_token_liquidity
    ) = swap(
        input_amount=_input['int_swap_amount'],
        input_token_liquidity=state_current['int_liquidity'][input_token_name],
        output_token_liquidity=state_current['int_liquidity'][output_token_name]
    )
    print("INT AMM: output %f %s" % (output_amount, output_token_name))
    
    # Second, swap ALT for FLASH needed in the staking process (executed in parallel)
    flash_needed = _input['flash_to_stake']
    print("flash_needed: ", flash_needed)
    
    # In this swap, the input is always ALT, so determine which updated liquidity
    # from the first swap should be used for input/output
    second_input_token_liquidity = updated_input_token_liquidity
    second_output_token_liquidity = updated_output_token_liquidity
    if input_token_name == "flash":
        second_input_token_liquidity = updated_output_token_liquidity
        second_output_token_liquidity = updated_input_token_liquidity
    
    (output_amount,
        updated_alt_token_liquidity,
        updated_flash_token_liquidity
    ) = swap(
        input_amount=flash_needed,
        input_token_liquidity=second_input_token_liquidity,
        output_token_liquidity=second_output_token_liquidity
    )
    print("INT AMM for STAKE: output %f flash" % (output_amount))
    
    # Finally, update the liquidity states after both swaps
    updated_int_liquidity = {
        'alt': updated_alt_token_liquidity,
        'flash': updated_flash_token_liquidity
    }
    return ('int_liquidity', updated_int_liquidity)

#####
# FLASH stake execution
# All new stakes are assumed to come from ALT swapped internally (and LPs gained) at the
# time of staking, not existing LP tokens passed for staking. All minted FLASH are also
# assumed to be immediately sold on the external CP AMM to ensure the market accurately
# reflects the new supply of FLASH on the market and the FPY is fulfilled.
def s_new_stakes(_params, substep, state_history, state_current, _input, **kwargs):
    flash_to_stake = _input['flash_to_stake']
    print("flash_to_stake: ", flash_to_stake)
    
    # The FLASH needed for staking should be swapped for internally in parallel
    # (see "s_int_swap") - we will assume the swap already occurred this timestep
    updated_stake_amt = state_current['staked'] + flash_to_stake
    print("updated_stake_amt: ", updated_stake_amt)
    
    return ('staked', updated_stake_amt)

## SETUP

In [49]:
del configs[:]
initial_conditions = {
    'market_apy': Decimal(0.01),
    'policies': {
        'fee_rate': Decimal(0.003), # Fee multiple
        'burn_rate': Decimal(0.01), # Burn multiple
        'mint_rate': Decimal(0.1) # The number of FLASH minted for each flashALT LP token staked PER DAY
    },
    'ext_liquidity': {
        'flash': Decimal(100000), # Denormalized amount in token units
        'alt': Decimal(100000) # Denormalized amount in token units
    },
    'int_liquidity': {
        'flash': Decimal(100000), # Denormalized amount in token units
        'alt': Decimal(100000) # Denormalized amount in token units
#         'providers': {
#             '0x000': 0
#         }
    },
    'staked': Decimal(0)
#     'minted': Decimal(0),
#     'stakes': [
#         {
#             'address': '0x001',
#             'amount':  Decimal(100), # Amount in token units
#             'length': 180 # Number of days selected to stake
#         }
#     ]
}

params = {
    'ext_swap_rand_factor': [100], # The multiple used to determine the amount of FLASH / ALT swapped in external exchanges (greater = more volatility)
    'market_apy_rand_range': [(10,25)], # The external rate of return is the return in the external market for a protocol of similar risk
}

MONTE_CARLO_RUNS = 2
SIMULATION_TIMESTEPS = range(10)
simulation_parameters = {
    'N': MONTE_CARLO_RUNS,
    'T': SIMULATION_TIMESTEPS,
    'M': params
}

In [50]:
partial_state_update_blocks = [
    { 
        'policies': {
            'market_apy': p_market_apy,
            'ext_swap_amount': p_ext_swap_amount,
            'ext_swap_input_token': p_ext_swap_input_token,
            'int_swap_amount': p_int_swap_amount,
            'int_swap_input_token': p_int_swap_input_token,
            'flash_to_stake': p_flash_to_stake
        },
        'variables': {
            'ext_liquidity': s_ext_swap,
            'int_liquidity': s_int_swap,
            'staked': s_new_stakes
        }
    }
]

## EXECUTION

In [51]:
sim_config = config_sim(simulation_parameters)

exp = Experiment()
exp.append_configs(sim_configs=sim_config, 
                   initial_state=initial_conditions,
                   partial_state_update_blocks=partial_state_update_blocks)

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.local_mode)
executor = Executor(exec_context=exec_context, configs=configs) 
(records, tensor_field, session) = executor.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (10, 2, 2, 5)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 0]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
EXT AMM: output 5.993113 flash
EXT AMM for STAKE: output 4606.619286 alt
INT AMM: output 0.000000 flash
flash_needed:  48284.83672191829616919975555
INT AMM for STAKE: output 32562.221323 flash
flash_to_stake:  48284.83672191829616919975555
updated_stake_amt:  48284.83672191829616919975555
EXT AMM: output 20.304234 flash
EXT AMM for STAKE: output 472.731916 alt
INT AMM: output 52885.462536 alt
flash_needed:  5218.102393690291088883401884
INT AMM for STAKE: output 5436.177764 flash
flash_to_stake:  5218.102393690291088883

In [39]:
df = pd.DataFrame(records)
pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.set_option("display.max_rows", 10, "display.max_columns", 10)
df.set_index(['simulation', 'run', 'timestep', 'substep'])

market_apy  \
simulation run timestep substep                                                      
0          1   0        0        0.01000000000000000020816681711721685132943093...   
               1        1        0.01000000000000000020816681711721685132943093...   
               2        1        0.01000000000000000020816681711721685132943093...   
               3        1        0.01000000000000000020816681711721685132943093...   
               4        1        0.01000000000000000020816681711721685132943093...   
               5        1        0.01000000000000000020816681711721685132943093...   
               6        1        0.01000000000000000020816681711721685132943093...   
               7        1        0.01000000000000000020816681711721685132943093...   
               8        1        0.01000000000000000020816681711721685132943093...   
               9        1        0.01000000000000000020816681711721685132943093...   
               10       1        0.01000000000000000020816681711721685132943093...   
           2   0        0        0.01000000000000000020816681711721685132943093...   
               1        1        0.01000000000000000020816681711721685132943093...   
               2        1        0.01000000000000000020816681711721685132943093...   
               3        1        0.01000000000000000020816681711721685132943093...   
               4        1        0.01000000000000000020816681711721685132943093...   
               5        1        0.01000000000000000020816681711721685132943093...   
               6        1        0.01000000000000000020816681711721685132943093...   
               7        1        0.01000000000000000020816681711721685132943093...   
               8        1        0.01000000000000000020816681711721685132943093...   
               9        1        0.01000000000000000020816681711721685132943093...   
               10       1        0.01000000000000000020816681711721685132943093...   

                                                                          policies  \
simulation run timestep substep                                                      
0          1   0        0        {'fee_rate': 0.0030000000000000000624500451351...   
               1        1        {'fee_rate': 0.0030000000000000000624500451351...   
               2        1        {'fee_rate': 0.0030000000000000000624500451351...   
               3        1        {'fee_rate': 0.0030000000000000000624500451351...   
               4        1        {'fee_rate': 0.0030000000000000000624500451351...   
               5        1        {'fee_rate': 0.0030000000000000000624500451351...   
               6        1        {'fee_rate': 0.0030000000000000000624500451351...   
               7        1        {'fee_rate': 0.0030000000000000000624500451351...   
               8        1        {'fee_rate': 0.0030000000000000000624500451351...   
               9        1        {'fee_rate': 0.0030000000000000000624500451351...   
               10       1        {'fee_rate': 0.0030000000000000000624500451351...   
           2   0        0        {'fee_rate': 0.0030000000000000000624500451351...   
               1        1        {'fee_rate': 0.0030000000000000000624500451351...   
               2        1        {'fee_rate': 0.0030000000000000000624500451351...   
               3        1        {'fee_rate': 0.0030000000000000000624500451351...   
               4        1        {'fee_rate': 0.0030000000000000000624500451351...   
               5        1        {'fee_rate': 0.0030000000000000000624500451351...   
               6        1        {'fee_rate': 0.0030000000000000000624500451351...   
               7        1        {'fee_rate': 0.0030000000000000000624500451351...   
               8        1        {'fee_rate': 0.0030000000000000000624500451351...   
               9        1        {'fee_rate': 0.0030000000000000000624500451351...   
               10       1    

## OUTPUT & GRAPHS

In [ ]:
for simulation_id in range(2):
    ax = None
    for i in range(0, MONTE_CARLO_RUNS):
        ax =  df[(df['simulation']==simulation_id) & (df['run']==i+1) & (df['timestep']>0)].plot(x='data', y=['px'], marker='o', markersize=2,
                    markeredgewidth=4, alpha=0.8, markerfacecolor='black',
                    linewidth=1, figsize=(12,8),  title="px vs. data", 
                    ylabel='px', grid=True, fillstyle='none',  
                    xticks=list(range(1+np.int64(np.max(df[(df['simulation']==simulation_id)]['data'])))), legend=None,
                    yticks=list(range(1+np.int64(np.max(df[(df['simulation']==simulation_id)]['px'])))), ax=ax);
#                     yticks=list(range(1+(df[(df['simulation']==simulation_id)]['px']).max())), ax=ax);